In [1]:
import numpy as np
import cvxpy as cp
from scipy.optimize import minimize 
import pandas as pd

### 1
零和 混合策略

In [2]:
A = np.array([[0, -1, 1], [1, 0, -1], [-1, 1, 0]])
x = cp.Variable(3, nonneg=True)
y = cp.Variable(3, nonneg=True)
u = cp.Variable()
v = cp.Variable()

obj1 = cp.Maximize(u)
cons1 = [
    A.T @ x >= u,   # aij*xi 每列的和 >= u
    cp.sum(x) == 1,
]
prob1 = cp.Problem(obj1, cons1)
prob1.solve(solver='GUROBI')
print("最优解 x：", x.value)
print("最优值 u：", prob1.value)

obj2 = cp.Minimize(v)
cons2 = [
    A @ y <= v,   # aij*yj 每行的和 <= v
    cp.sum(y) == 1,
]
prob2 = cp.Problem(obj2, cons2)
prob2.solve(solver='GUROBI')
print("最优解 y：", y.value)
print("最优值 v：", prob2.value)

最优解 x： [0.33333333 0.33333333 0.33333333]
最优值 u： 0.0
最优解 y： [0.33333333 0.33333333 0.33333333]
最优值 v： 0.0


### 2
零和 混合策略

In [3]:
A = np.array([[0.58, 0.95], [0.93, 0.70]])
x = cp.Variable(2, nonneg=True)
y = cp.Variable(2, nonneg=True)
u = cp.Variable()
v = cp.Variable()

obj1 = cp.Maximize(u)
cons1 = [
    A.T @ x >= u,   # aij*xi 每列的和 >= u
    cp.sum(x) == 1,
]
prob1 = cp.Problem(obj1, cons1)
prob1.solve(solver='GUROBI')
print("最优解 x：", x.value)    # 罚球队员
print("最优值 u：", prob1.value)

obj2 = cp.Minimize(v)
cons2 = [
    A @ y <= v,   # aij*yj 每行的和 <= v
    cp.sum(y) == 1,
]
prob2 = cp.Problem(obj2, cons2)
prob2.solve(solver='GUROBI')
print("最优解 y：", y.value)    # 守门队员
print("最优值 v：", prob2.value)

最优解 x： [0.38333333 0.61666667]
最优值 u： 0.7958333333333334
最优解 y： [0.41666667 0.58333333]
最优值 v： 0.7958333333333334


### 3
非零和 混合策略 囚徒困境

In [4]:
A = np.array([[2, 5], [0, 4]])
B = np.array([[2, 0], [5, 4]])

def obj(x):
    return np.sum(x)

def ineq(x):
    xs = x[:2]
    ys = x[2:]
    return np.array([
        -(A @ ys - xs @ A @ ys),
        -(B.T @ xs - xs @ B @ ys),
    ]).flatten()

def eq(x):
    xs = x[:2]
    ys = x[2:]
    return np.array([
        np.sum(xs) - 1,
        np.sum(ys) - 1,
    ]).flatten()

x0 = np.ones(4)/2
cons = [
    {'type': 'ineq', 'fun': ineq},
    {'type': 'eq', 'fun': eq},
]
bd = [(0, 1)]*4
ret = minimize(obj, x0, constraints=cons, bounds=bd)
print(ret)
print('-'*80)
print("最优值为：", ret.fun)
print("最优解为：", ret.x.round(4))

x_r = ret.x[:2]
y_r = ret.x[2:]
x_score = x_r @ A @ y_r
y_score = x_r @ B @ y_r
print()
print("国家Ⅰ得分的期望：", x_score)
print("国家Ⅱ得分的期望：", y_score)

     fun: 2.0
     jac: array([1., 1., 1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 6
     nit: 1
    njev: 1
  status: 0
 success: True
       x: array([1.00000000e+00, 1.23456800e-13, 1.00000000e+00, 1.35946809e-12])
--------------------------------------------------------------------------------
最优值为： 2.0
最优解为： [1. 0. 1. 0.]

国家Ⅰ得分的期望： 2.000000000003831
国家Ⅱ得分的期望： 1.9999999999976512


可以看到两国的最优策略都是扩军，此为囚徒困境一实例

### 4
相亲配对问题

In [5]:
df1 = pd.read_excel('../../../hw/16第16章  博弈论习题解答/ti16_4.xlsx', header=None, sheet_name='Sheet1')
df2 = pd.read_excel('../../../hw/16第16章  博弈论习题解答/ti16_4.xlsx', header=None, sheet_name='Sheet2')
B = df1.values
G = df2.values
B_ability, B_age, B_expect = B[:, :5], B[:, 5], B[:, 6:]
G_ability, G_age, G_expect = G[:, :5], G[:, 5], G[:, 6:]

n, m = len(B), len(G)
A = np.zeros((n, m))    # 是否能够配对
for i in range(n):
    for j in range(m):
        if B_age[i] - G_age[j] <= 5 and G_age[j] - B_age[i] <= 2:
            b_abi, b_exp = B_ability[i], B_expect[i]
            g_abi, g_exp = G_ability[j], G_expect[j]
            if np.sum(b_abi >= g_exp) >= 2 and np.sum(g_abi >= b_exp) >= 2:
                A[i, j] = 1
A

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 0.],
       [1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 0.],
       [1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.],
       [0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
        1., 1., 0., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.],
       [1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
        1., 1., 0., 1.],
       [1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1.,
        1., 1., 1., 1.],
       [1., 0., 0., 1., 0., 0.

In [6]:
x = cp.Variable((n, m), boolean=True)
obj = cp.Maximize(cp.sum(cp.multiply(A, x)))
cons = [
    cp.sum(x, axis=0) == 1,
    cp.sum(x, axis=1) == 1,
]
prob = cp.Problem(obj, cons)
prob.solve(solver="GUROBI")
print("最优解：", end='\n')
x_r, y_r = np.nonzero(x.value)
print(x_r+1); print(y_r+1)
print("最优值：", prob.value)

最优解：
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[ 5  3  1 12 14 17  2  9 13  4 15  7 18 10 16  8 19  6 11 20]
最优值： 20.0


满意度最高（符合条件最多）

In [7]:
B = np.zeros((n, m))    # 满意度（简单地以满足条件数来估计，但可以采用模糊评价法）
for i in range(n):
    for j in range(m):
        if B_age[i] - G_age[j] <= 5 and G_age[j] - B_age[i] <= 2:
            b_abi, b_exp = B_ability[i], B_expect[i]
            g_abi, g_exp = G_ability[j], G_expect[j]
            if np.sum(b_abi >= g_exp) >= 2 and np.sum(g_abi >= b_exp) >= 2:
                B[i, j] = np.sum(b_abi >= g_exp) + np.sum(g_abi >= b_exp)
B

array([[6., 7., 5., 6., 5., 6., 5., 6., 6., 4., 5., 5., 8., 5., 6., 6.,
        5., 8., 5., 4.],
       [6., 8., 5., 5., 5., 6., 6., 6., 7., 5., 6., 6., 7., 6., 6., 6.,
        5., 9., 5., 0.],
       [5., 7., 6., 0., 6., 6., 6., 0., 7., 7., 7., 5., 7., 7., 7., 6.,
        4., 7., 5., 0.],
       [5., 8., 6., 7., 5., 7., 5., 0., 8., 0., 7., 6., 7., 6., 7., 6.,
        4., 8., 5., 5.],
       [4., 8., 7., 6., 6., 5., 6., 6., 7., 6., 7., 4., 5., 6., 7., 5.,
        5., 7., 6., 6.],
       [0., 7., 6., 0., 7., 6., 6., 0., 7., 5., 9., 5., 7., 7., 6., 0.,
        5., 7., 0., 6.],
       [6., 7., 7., 6., 5., 7., 6., 6., 7., 6., 8., 5., 6., 6., 7., 6.,
        6., 8., 6., 5.],
       [7., 7., 6., 7., 4., 7., 5., 7., 7., 4., 7., 5., 8., 7., 7., 6.,
        4., 8., 6., 6.],
       [4., 4., 5., 0., 0., 5., 4., 0., 0., 4., 5., 0., 5., 4., 5., 0.,
        5., 6., 0., 4.],
       [6., 7., 6., 7., 0., 6., 6., 0., 9., 6., 8., 0., 6., 6., 7., 6.,
        6., 7., 7., 8.],
       [7., 0., 0., 6., 0., 0.

In [8]:
x = cp.Variable((n, m), boolean=True)
obj = cp.Maximize(cp.sum(cp.multiply(B, x)))
cons = [
    cp.sum(x, axis=0) == 1,
    cp.sum(x, axis=1) == 1,
]
prob = cp.Problem(obj, cons)
prob.solve(solver="GUROBI")
print("最优解：", end='\n')
x_r, y_r = np.nonzero(x.value)
print(x_r+1); print(y_r+1)
print("最优值：", prob.value)   # 最佳方案

最优解：
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[13 12 14  4  2 11  3 19 17 20  1  5  7 16  6  9 18 15  8 10]
最优值： 145.0


In [9]:
print(B[x_r, y_r])  # 各对符合条件数

[8. 6. 7. 7. 8. 9. 7. 6. 5. 8. 7. 7. 7. 8. 7. 8. 9. 7. 8. 6.]
